In [110]:
import pandas as pd
import dill
from collections import defaultdict
from datetime import datetime, timedelta
from statsmodels.tsa.arima.model import ARIMA
from pmdarima import auto_arima

In [105]:
N_PRIOR_DAYS = 502

In [111]:
tickers = set(dill.load(open('/home/flor/cs236project/dividend_tickers.pkd','rb')))
dill.dump(tickers,open('dividend_tickers.pkd','wb'))
print(len(tickers))
tickers

110


{'A',
 'ACN',
 'AMGN',
 'AMP',
 'AMT',
 'AON',
 'AOS',
 'APD',
 'AVGO',
 'AWK',
 'BBY',
 'BLK',
 'CAG',
 'CAT',
 'CB',
 'CE',
 'CF',
 'CI',
 'CLX',
 'CMCSA',
 'CME',
 'COF',
 'COO',
 'COP',
 'COST',
 'CPB',
 'DG',
 'DGX',
 'DHI',
 'DHR',
 'ECL',
 'EFX',
 'EG',
 'EOG',
 'EQIX',
 'EQR',
 'ESS',
 'F',
 'FAST',
 'FDS',
 'FDX',
 'GD',
 'GL',
 'GS',
 'GWW',
 'HD',
 'HII',
 'HRL',
 'HUM',
 'IFF',
 'INTU',
 'J',
 'JPM',
 'KEY',
 'KHC',
 'LEN',
 'LIN',
 'LMT',
 'LRCX',
 'MA',
 'MAA',
 'MKTX',
 'MLM',
 'MMC',
 'MPC',
 'MSCI',
 'NDSN',
 'NEE',
 'NEM',
 'NOC',
 'NSC',
 'NVDA',
 'ODFL',
 'OKE',
 'ORCL',
 'PGR',
 'PKG',
 'PM',
 'PNC',
 'POOL',
 'PSX',
 'PXD',
 'REG',
 'RMD',
 'ROK',
 'ROP',
 'RVTY',
 'SBAC',
 'SNA',
 'SPGI',
 'SPY',
 'STX',
 'STZ',
 'SWK',
 'SWKS',
 'SYF',
 'TECH',
 'TFX',
 'TMO',
 'TSCO',
 'TT',
 'TXN',
 'UNH',
 'UNP',
 'VLO',
 'VMC',
 'WHR',
 'WST',
 'WYNN',
 'XEL'}

In [26]:
def get_ticker_dates(tickers):
    d_daily_data = dill.load(open('d_all_EOD.pkd','rb'))
    d_ticker_dates = {}

    for ticker in tickers:
        df = d_daily_data[ticker]
        dates = df[df.divCash > 0].date.values[-20:]
        dates = [datetime.strptime(str(x)[:10],'%Y-%m-%d') for x in dates]
        d_ticker_dates[ticker] = dates
    return d_ticker_dates
ticker_dates = get_ticker_dates(tickers)
dill.dump(ticker_dates,open('ticker_dates.pkd','wb'))

In [5]:
d_daily_data = dill.load(open('d_all_EOD.pkd','rb'))

In [72]:
def find_splits(df, date):
    df = df.reset_index(drop = True)
    df = df[df.date <= date]
    df_splits = df[df.splitFactor != 1]
    last_split_index = 0
    if len(df_splits) > 0:
        last_split_index = df_splits.index[-1]
    return len(df) - last_split_index    

min_days = 1000000
for ticker, dates in ticker_dates.items():
    for date in dates:
        n_days = find_splits(d_daily_data[ticker],date)
        if n_days > 0 and n_days < N_PRIOR_DAYS:
            print(date, ticker, n_days)
        min_days = min(min_days, n_days)
        
min_days

2019-07-24 00:00:00 FAST 43
2019-10-24 00:00:00 FAST 108
2020-01-30 00:00:00 FAST 174
2020-04-27 00:00:00 FAST 234
2020-07-27 00:00:00 FAST 297
2020-10-26 00:00:00 FAST 361
2020-12-01 00:00:00 FAST 386
2021-02-02 00:00:00 FAST 428
2021-04-23 00:00:00 FAST 484
2020-11-25 00:00:00 NEE 22
2021-02-25 00:00:00 NEE 83
2021-06-01 00:00:00 NEE 149
2021-08-26 00:00:00 NEE 210
2021-11-24 00:00:00 NEE 273
2022-02-28 00:00:00 NEE 337
2022-05-27 00:00:00 NEE 400
2022-08-29 00:00:00 NEE 463
2019-01-24 00:00:00 LEN 302
2019-04-23 00:00:00 LEN 363
2019-07-10 00:00:00 LEN 417
2019-10-17 00:00:00 LEN 487
2020-12-03 00:00:00 NVDA 8
2021-03-09 00:00:00 NVDA 72
2021-06-09 00:00:00 NVDA 136
2021-08-31 00:00:00 NVDA 31
2021-12-01 00:00:00 NVDA 95
2022-03-02 00:00:00 NVDA 157
2022-06-08 00:00:00 NVDA 225
2022-09-07 00:00:00 NVDA 287
2022-11-30 00:00:00 NVDA 346
2023-03-07 00:00:00 NVDA 411
2023-06-07 00:00:00 NVDA 475
2018-12-31 00:00:00 CMCSA 469
2023-02-10 00:00:00 TECH 50
2023-05-12 00:00:00 TECH 113
2023-

8

In [112]:
### create arima datasets

def adjust_splits(df):
    df = df.reset_index(drop = True)
    splits = [(datetime.strptime(str(x[0])[:10],'%Y-%m-%d'),x[1]) for x in df[df.splitFactor != 1][['date','splitFactor']].to_records(index=False)]
    df = df[['date','close','high','low']].set_index('date')
    for date, factor in splits:
        df.loc[df.index < date] = df.loc[df.index < date] / factor
    return df

d_dfs = defaultdict(dict)
for ticker, dates in ticker_dates.items():
    df = adjust_splits(d_daily_data[ticker])
    
    for date in dates:
        df1 = df.loc[df.index <= date][-N_PRIOR_DAYS:]
        if len(df1) >= N_PRIOR_DAYS:
            d_dfs[ticker][date] = df1

In [113]:
dill.dump(d_dfs,open('d_dfs.pkd','wb'))

In [ ]:
def get_model_predict():
    d_models = defaultdict(dict)
    d_predictions = defaultdict(dict)
    for ticker, dates in d_dfs.items():
        for date, df in dates.items():
            params_low = auto_arima(df.low).order
            params_high = auto_arima(df.high).order
            d_temp_models = {}
            d_temp_predict = {}
            d_temp_models['low'] = ARIMA(df['low'].values[:-2], order=params_low).fit()
            d_temp_models['high'] = ARIMA(df['high'].values, order=params_high).fit()
            
            d_temp_predict['low'] = d_temp_models['low'].predict(start=len(df)-2,end=len(df)-2)[0]
            d_temp_predict['high'] = d_temp_models['high'].predict(start=len(df),end=len(df))[0]
            
            d_models[ticker][date] = d_temp_models
            d_predictions[ticker][date] = d_temp_predict
            print(ticker,date)
            print(params_low,params_high)
            print(df.tail(3))
            print(d_temp_predict)
            dill.dump(d_models,open('arima_models.pkd','wb'))
            dill.dump(d_predictions,open('arima_predictions.pkd','wb'))
            
get_model_predict()

AWK 2018-11-08 00:00:00
(0, 1, 4) (2, 1, 2)
            close   high    low
date                           
2018-11-06  89.10  89.17  88.02
2018-11-07  90.09  90.36  89.47
2018-11-08  88.89  90.16  88.32
{'low': 88.07831306253938, 'high': 90.0202573927387}
AWK 2019-02-06 00:00:00
(2, 1, 2) (2, 1, 2)
            close   high    low
date                           
2019-02-04  95.58  95.58  93.90
2019-02-05  95.86  96.15  94.85
2019-02-06  95.15  95.79  94.72
{'low': 93.7774683392823, 'high': 95.74672911371819}
AWK 2019-05-10 00:00:00
(2, 1, 2) (2, 1, 2)
             close    high     low
date                              
2019-05-08  105.86  107.27  105.73
2019-05-09  106.28  106.47  105.38
2019-05-10  108.27  108.30  105.56
{'low': 105.58885899893374, 'high': 108.56348533723782}
AWK 2019-08-08 00:00:00
(2, 1, 2) (2, 1, 2)
             close    high     low
date                              
2019-08-06  117.42  117.69  114.45
2019-08-07  118.45  119.26  116.41
2019-08-08  119.41  119.95 

/home/flor/anaconda3/lib/python3.6/site-packages/statsmodels/tsa/statespace/sarimax.py:963: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/home/flor/anaconda3/lib/python3.6/site-packages/statsmodels/tsa/statespace/sarimax.py:975: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


AWK 2020-05-11 00:00:00
(3, 1, 2) (4, 1, 3)
             close    high     low
date                              
2020-05-07  119.08  123.52  118.39
2020-05-08  118.03  120.31  117.52
2020-05-11  119.49  119.82  115.55
{'low': 119.33290983389237, 'high': 119.51312390518409}


/home/flor/anaconda3/lib/python3.6/site-packages/statsmodels/tsa/statespace/sarimax.py:963: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/home/flor/anaconda3/lib/python3.6/site-packages/statsmodels/tsa/statespace/sarimax.py:975: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


AWK 2020-08-11 00:00:00
(3, 1, 2) (3, 1, 3)
             close     high     low
date                               
2020-08-07  149.79  150.035  146.09
2020-08-10  149.11  150.470  148.44
2020-08-11  142.42  148.495  141.71
{'low': 146.61982412761972, 'high': 147.93378169821776}


...

AWK 2023-02-06 00:00:00
(1, 1, 0) (5, 1, 3)
             close    high      low
date                               
2023-02-02  160.60  162.59  158.030
2023-02-03  156.61  159.95  155.100
2023-02-06  156.41  156.50  154.785
{'low': 158.55291569885392, 'high': 155.69809989754208}


/home/flor/anaconda3/lib/python3.6/site-packages/statsmodels/tsa/statespace/sarimax.py:963: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/home/flor/anaconda3/lib/python3.6/site-packages/statsmodels/tsa/statespace/sarimax.py:975: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/home/flor/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


AWK 2023-05-08 00:00:00
(1, 1, 0) (5, 1, 3)
             close    high     low
date                              
2023-05-04  147.82  148.38  146.07
2023-05-05  148.43  148.90  147.12
2023-05-08  147.47  148.82  146.32
{'low': 146.0348817840462, 'high': 148.47098313035576}


/home/flor/anaconda3/lib/python3.6/site-packages/statsmodels/tsa/statespace/sarimax.py:963: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/home/flor/anaconda3/lib/python3.6/site-packages/statsmodels/tsa/statespace/sarimax.py:975: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/home/flor/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


AWK 2023-08-07 00:00:00
(1, 1, 0) (5, 1, 3)
             close      high     low
date                                
2023-08-03  142.13  145.2825  142.13
2023-08-04  141.97  143.1500  141.20
2023-08-07  142.11  142.9000  141.54
{'low': 141.87740426640858, 'high': 142.9556557999038}
GWW 2018-11-08 00:00:00
(0, 1, 0) (0, 1, 1)
             close    high     low
date                              
2018-11-06  287.58  289.81  282.86
2018-11-07  296.02  296.79  287.90
2018-11-08  293.02  296.27  292.07
{'low': 282.86, 'high': 296.16974777699795}
GWW 2019-02-08 00:00:00
(0, 1, 0) (0, 1, 1)
             close    high     low
date                              
2019-02-06  303.52  305.64  298.72
2019-02-07  303.06  306.50  300.72
2019-02-08  300.43  302.35  298.58
{'low': 298.72, 'high': 302.0357153212779}
GWW 2019-05-10 00:00:00
(0, 1, 0) (0, 1, 1)
             close    high     low
date                              
2019-05-08  276.57  278.79  273.86
2019-05-09  273.44  274.69  267.65
2019-05

In [108]:
### find optimal parameters
def get_params(col):
    params = auto_arima(col,seasonal=True)
    return {'order':params.order, 'seasonal_order': params.seasonal_order}

In [109]:
d_params = defaultdict(dict)
for ticker, dates in d_dfs.items():
    for date,df in dates.items():
        d = {}
        for col in ['high','low','close']:
            d[col] = get_params(df[col])
        d_params[ticker][date] = d
        dill.dump(d_params,open('arima_params.pkd','wb'))

/home/flor/anaconda3/lib/python3.6/site-packages/pmdarima/arima/_auto_solvers.py:522: ModelFitWarning: Error fitting  ARIMA(2,1,2)(0,0,0)[0] intercept (if you do not want to see these warnings, run with error_action="ignore").
Traceback:
Traceback (most recent call last):
  File "/home/flor/anaconda3/lib/python3.6/site-packages/pmdarima/arima/_auto_solvers.py", line 506, in _fit_candidate_model
    fit.fit(y, X=X, **fit_params)
  File "/home/flor/anaconda3/lib/python3.6/site-packages/pmdarima/arima/arima.py", line 597, in fit
    self._fit(y, X, **fit_args)
  File "/home/flor/anaconda3/lib/python3.6/site-packages/pmdarima/arima/arima.py", line 518, in _fit
    fit, self.arima_res_ = _fit_wrapper()
  File "/home/flor/anaconda3/lib/python3.6/site-packages/pmdarima/arima/arima.py", line 512, in _fit_wrapper
    **fit_args)
  File "/home/flor/anaconda3/lib/python3.6/site-packages/statsmodels/tsa/statespace/mlemodel.py", line 659, in fit
    skip_hessian=True, **kwargs)
  File "/home/flor/a

/home/flor/anaconda3/lib/python3.6/site-packages/pmdarima/arima/_auto_solvers.py:522: ModelFitWarning: Error fitting  ARIMA(2,1,2)(0,0,0)[0] intercept (if you do not want to see these warnings, run with error_action="ignore").
Traceback:
Traceback (most recent call last):
  File "/home/flor/anaconda3/lib/python3.6/site-packages/pmdarima/arima/_auto_solvers.py", line 506, in _fit_candidate_model
    fit.fit(y, X=X, **fit_params)
  File "/home/flor/anaconda3/lib/python3.6/site-packages/pmdarima/arima/arima.py", line 597, in fit
    self._fit(y, X, **fit_args)
  File "/home/flor/anaconda3/lib/python3.6/site-packages/pmdarima/arima/arima.py", line 518, in _fit
    fit, self.arima_res_ = _fit_wrapper()
  File "/home/flor/anaconda3/lib/python3.6/site-packages/pmdarima/arima/arima.py", line 512, in _fit_wrapper
    **fit_args)
  File "/home/flor/anaconda3/lib/python3.6/site-packages/statsmodels/tsa/statespace/mlemodel.py", line 659, in fit
    skip_hessian=True, **kwargs)
  File "/home/flor/a

/home/flor/anaconda3/lib/python3.6/site-packages/pmdarima/arima/_auto_solvers.py:522: ModelFitWarning: Error fitting  ARIMA(2,1,2)(0,0,0)[0] intercept (if you do not want to see these warnings, run with error_action="ignore").
Traceback:
Traceback (most recent call last):
  File "/home/flor/anaconda3/lib/python3.6/site-packages/pmdarima/arima/_auto_solvers.py", line 506, in _fit_candidate_model
    fit.fit(y, X=X, **fit_params)
  File "/home/flor/anaconda3/lib/python3.6/site-packages/pmdarima/arima/arima.py", line 597, in fit
    self._fit(y, X, **fit_args)
  File "/home/flor/anaconda3/lib/python3.6/site-packages/pmdarima/arima/arima.py", line 518, in _fit
    fit, self.arima_res_ = _fit_wrapper()
  File "/home/flor/anaconda3/lib/python3.6/site-packages/pmdarima/arima/arima.py", line 512, in _fit_wrapper
    **fit_args)
  File "/home/flor/anaconda3/lib/python3.6/site-packages/statsmodels/tsa/statespace/mlemodel.py", line 659, in fit
    skip_hessian=True, **kwargs)
  File "/home/flor/a

/home/flor/anaconda3/lib/python3.6/site-packages/pmdarima/arima/_auto_solvers.py:522: ModelFitWarning: Error fitting  ARIMA(2,1,2)(0,0,0)[0] intercept (if you do not want to see these warnings, run with error_action="ignore").
Traceback:
Traceback (most recent call last):
  File "/home/flor/anaconda3/lib/python3.6/site-packages/pmdarima/arima/_auto_solvers.py", line 506, in _fit_candidate_model
    fit.fit(y, X=X, **fit_params)
  File "/home/flor/anaconda3/lib/python3.6/site-packages/pmdarima/arima/arima.py", line 597, in fit
    self._fit(y, X, **fit_args)
  File "/home/flor/anaconda3/lib/python3.6/site-packages/pmdarima/arima/arima.py", line 518, in _fit
    fit, self.arima_res_ = _fit_wrapper()
  File "/home/flor/anaconda3/lib/python3.6/site-packages/pmdarima/arima/arima.py", line 512, in _fit_wrapper
    **fit_args)
  File "/home/flor/anaconda3/lib/python3.6/site-packages/statsmodels/tsa/statespace/mlemodel.py", line 659, in fit
    skip_hessian=True, **kwargs)
  File "/home/flor/a

/home/flor/anaconda3/lib/python3.6/site-packages/pmdarima/arima/_auto_solvers.py:522: ModelFitWarning: Error fitting  ARIMA(2,1,2)(0,0,0)[0] intercept (if you do not want to see these warnings, run with error_action="ignore").
Traceback:
Traceback (most recent call last):
  File "/home/flor/anaconda3/lib/python3.6/site-packages/pmdarima/arima/_auto_solvers.py", line 506, in _fit_candidate_model
    fit.fit(y, X=X, **fit_params)
  File "/home/flor/anaconda3/lib/python3.6/site-packages/pmdarima/arima/arima.py", line 597, in fit
    self._fit(y, X, **fit_args)
  File "/home/flor/anaconda3/lib/python3.6/site-packages/pmdarima/arima/arima.py", line 518, in _fit
    fit, self.arima_res_ = _fit_wrapper()
  File "/home/flor/anaconda3/lib/python3.6/site-packages/pmdarima/arima/arima.py", line 512, in _fit_wrapper
    **fit_args)
  File "/home/flor/anaconda3/lib/python3.6/site-packages/statsmodels/tsa/statespace/mlemodel.py", line 659, in fit
    skip_hessian=True, **kwargs)
  File "/home/flor/a

/home/flor/anaconda3/lib/python3.6/site-packages/pmdarima/arima/_auto_solvers.py:522: ModelFitWarning: Error fitting  ARIMA(2,1,2)(0,0,0)[0] intercept (if you do not want to see these warnings, run with error_action="ignore").
Traceback:
Traceback (most recent call last):
  File "/home/flor/anaconda3/lib/python3.6/site-packages/pmdarima/arima/_auto_solvers.py", line 506, in _fit_candidate_model
    fit.fit(y, X=X, **fit_params)
  File "/home/flor/anaconda3/lib/python3.6/site-packages/pmdarima/arima/arima.py", line 597, in fit
    self._fit(y, X, **fit_args)
  File "/home/flor/anaconda3/lib/python3.6/site-packages/pmdarima/arima/arima.py", line 518, in _fit
    fit, self.arima_res_ = _fit_wrapper()
  File "/home/flor/anaconda3/lib/python3.6/site-packages/pmdarima/arima/arima.py", line 512, in _fit_wrapper
    **fit_args)
  File "/home/flor/anaconda3/lib/python3.6/site-packages/statsmodels/tsa/statespace/mlemodel.py", line 659, in fit
    skip_hessian=True, **kwargs)
  File "/home/flor/a

/home/flor/anaconda3/lib/python3.6/site-packages/pmdarima/arima/_auto_solvers.py:522: ModelFitWarning: Error fitting  ARIMA(2,1,2)(0,0,0)[0] intercept (if you do not want to see these warnings, run with error_action="ignore").
Traceback:
Traceback (most recent call last):
  File "/home/flor/anaconda3/lib/python3.6/site-packages/pmdarima/arima/_auto_solvers.py", line 506, in _fit_candidate_model
    fit.fit(y, X=X, **fit_params)
  File "/home/flor/anaconda3/lib/python3.6/site-packages/pmdarima/arima/arima.py", line 597, in fit
    self._fit(y, X, **fit_args)
  File "/home/flor/anaconda3/lib/python3.6/site-packages/pmdarima/arima/arima.py", line 518, in _fit
    fit, self.arima_res_ = _fit_wrapper()
  File "/home/flor/anaconda3/lib/python3.6/site-packages/pmdarima/arima/arima.py", line 512, in _fit_wrapper
    **fit_args)
  File "/home/flor/anaconda3/lib/python3.6/site-packages/statsmodels/tsa/statespace/mlemodel.py", line 659, in fit
    skip_hessian=True, **kwargs)
  File "/home/flor/a

KeyboardInterrupt: 